# 1. Introduction about dataset
## Context
* The **SMS Spam Collection** is a set of SMS tagged messages that have been collected for SMS Spam research.
* It contains one set of SMS messages in English of **5,574** messages, tagged acording being **ham** (legitimate) or **spam.**

## Content
* The files contain one message per line.
* Each line is composed by two columns:
 * v1 contains the label (ham or spam) and
 * v2 contains the raw text.




**Thanks to:-**
* [SMS Spam Collection](https://archive.ics.uci.edu/dataset/228/sms+spam+collection)

# 2. Import libraries
- To building a **spam classifier** we need to perform **text preprocessing.**
- For this required necessary libraries and tools to import are
  - data manipulation,
  - text cleaning,
  - stemming,
  - lemmatization, and
  - removal of common stopwords.

These steps help to prepare raw text data for machine learning models, improving their accuracy and performance.

In [1]:
# provides tools for text preprocessing, tokenization, stemming, lemmatization, stopwords removal, and more.
import nltk

# Helps in loading, analyzing, and manipulating the dataset efficiently. It is typically used to read the data, handle missing values, and structure the data in a tabular format, particularly when working with structured data like CSV files.
import pandas as pd

# for cleaning text data, such as removing special characters, punctuations, and other unwanted patterns from the text.
import re

# simplify words in text data to their base form, which helps in reducing the dimensionality of text data and improving the performance of the model.
from nltk.stem import PorterStemmer

# convert words to their base or dictionary form (e.g., "better" to "good"). It is more context-aware than stemming, making it more effective in NLP tasks.
from nltk.stem import WordNetLemmatizer

# removing the frequently occurring words (e.g., "the", "and", "is") that do not contribute significantly to the classification task, thereby improving model efficiency and accuracy.
from nltk.corpus import stopwords

# The TfidfVectorizer converts the text data into a matrix of TF-IDF features.
from sklearn.feature_extraction.text import TfidfVectorizer

# scikit learn library to split arrays or matrices into random train and test subsets.
from sklearn.model_selection import train_test_split

# The multinomial Naive Bayes classifier is suitable for classification with discrete features (e.g., word counts for text classification). The multinomial distribution normally requires integer feature counts. However, in practice, fractional counts such as tf-idf may also work.
from sklearn.naive_bayes import MultinomialNB

# sklearn functions to perform accuracy classification score, main classification metrics and compute confusion matrix to evaluate the accuracy of a classification.
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 3. Load the Dataset

In [2]:
# define the URL, points to a CSV file hosted on GitHub, specifically at the repository and path where the dataset is stored.
url_sms ='https://raw.githubusercontent.com/akdubey2k/NLP/main/Spam_Classifier/SMSSpamCollection.csv'

# 4. Dataset EDA (Exploratory Data Analysis)
## 4.1 Read the data from **github** repositary using **pandas**.

In [3]:
# url_new="https://raw.githubusercontent.com/akdubey2k/NLP/main/Spam_Classifier/SpamCollection.csv"

# Loading the CSV file from the URL into a pandas DataFrame without header
df = pd.read_csv(url_sms, sep='\t')
df.head()

,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
0,ham,Ok lar... Joking wif u oni...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,ham,U dun say so early hor... U c already then say...
3,ham,"Nah I don't think he goes to usf, he lives aro..."
4,spam,FreeMsg Hey there darling it's been 3 week's n...


## 4.2 Renaming columns for clarity
#### In a **Pandas** dataframe by inserting new columns named as **label**, **message** and **index** by default true.

* The first column contains the labels (
  * **ham** for non-spam and
  * **spam** for spam).
* The second column contains the message text.

In [4]:
df = pd.read_csv(url_sms, sep='\t', names=['label', 'message'])
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## 4.3 Mapping labels to numerical values:
### spam : 1
### ham :  0


In [5]:
df['label'] = df['label'].map({'spam': 1, 'ham': 0})
df.head()

,label,message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


# 5. Dataset distribution into training and test set
#### Spliting the dataset into **training** and **test** set, using **sklearn**

In [6]:
X_train, X_test, y_train, y_test=train_test_split(df['message'], df['label'], test_size=0.20, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(4457,)
(1115,)
(4457,)
(1115,)


# 6. Cleaning and Preprocessing of data

In [7]:
# Vectorizing the text messages using TF-IDF
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

print(X_train_tfidf.shape)
print(X_test_tfidf.shape)

(4457, 7702)
(1115, 7702)


# 7. Building the Naive Bayes Model

In [8]:
# build a model using a machine learning algorithm like Naive Bayes.
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

MultinomialNB()

# 8. Model prediction on test set

In [9]:
# Making predictions on the test set
y_pred = model.predict(X_test_tfidf)
y_pred
accuracy = model.score(X_test_tfidf, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.9668161434977578


# 9. Model evaluation

In [10]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)

Accuracy: 0.9668161434977578
Classification Report:
               precision    recall  f1-score   support

           0       0.96      1.00      0.98       966
           1       1.00      0.75      0.86       149

    accuracy                           0.97      1115
   macro avg       0.98      0.88      0.92      1115
weighted avg       0.97      0.97      0.96      1115

Confusion Matrix:
 [[966   0]
 [ 37 112]]


#### Creation of **Stemming** and **Lemmatization** object

In [11]:
ps = PorterStemmer()
wnl = WordNetLemmatizer()

#### Data **Cleaning** and **preprocessing**

In [12]:
corpus=[]
for i in range(0, len(df)):
  data=re.sub('[^a-zA-Z]', ' ', df['message'][i])
  # print(data)
  data=data.lower()
  data=data.split()
  #data=[ps.stem(word) for word in data if word not in set(stopwords.words('english'))]
  data=[wnl.lemmatize(word) for word in data if word not in set(stopwords.words('english'))]
  data=' '.join(data)
  corpus.append(data)

# print(corpus)

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


#### **Bag of word** creation for spam classification

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500)
X = cv.fit_transform(corpus).toarray()
X

#### get **dummies** from **pandas** dataframe for the **output/dependent** variable (**label** column)

In [ ]:
y = pd.get_dummies(df['label'])
y

#### As there is only two dummy varibales, so we do not need two column... we can convert this **dummy variable trap** by using single column.

In [ ]:
y = y.iloc[:,1].values # all rows and only one column.... values need to be present
y

#### Classification (Training) model using **Naive Bayes Classifier**


In [ ]:
from sklearn.naive_bayes import MultinomialNB
mtnb = MultinomialNB()
mtnb.fit(X, y)

In [ ]:
mtnb=mtnb.fit(X_train, y_train)
y_pred=mtnb.predict(X_test)
y_pred

#### Classification between **y_test** and **y_pred** values using **Confusion Matrix**

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, y_pred) # 2 x 2 matrix
cm

#### Check the accuracy score of model

In [ ]:
from sklearn.metrics import accuracy_score
ac=accuracy_score(y_test, y_pred)
ac